<a href="https://colab.research.google.com/github/LizCarter492/EnvDatSci/blob/main/CodeSprints/ObservationDependence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Additional packages required in Google Collaboratory:
!pip install pysal
!pip install libpysal
!pip insall geopandas

<div align="center">
<h2 style="text-align: center;">Moran's I for diagnosing spatial autocorrelation in model residuals.

In this example, we calculate a global Moran's I statistic to evaluate spatial autocorrelation in a dataset containing county-level results from the 2012 and 2016 presidential elections and US census data. We will then attempt to fit a linear regression predicting change in 2012-2016 voting patterns using two spatial covariates (county level per-capita income, and land area). We will evaluate the OLS linear regression residuals for spatial autocorrealtion,  then reevalute model fit using an autoregressive model.





In [ ]:
import pysal as ps
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from libpysal.weights.contiguity import Queen
import libpysal
from splot.libpysal import plot_spatial_weights
from splot.esda import moran_scatterplot
from splot.esda import plot_moran
from esda.moran import Moran_Local
from esda.moran import Moran
from statsmodels.api import OLS
from pysal.model import spreg
import urllib.request
import shutil
from mlxtend.preprocessing import standardize
sns.set_style('white')

First, we're going to open 'Elections' dataset from the Geodata Center at the University of Chicago.

* More on spatial data science resources from UC: https://spatial.uchicago.edu/
* A list of datasets available through lipysal: https://geodacenter.github.io/data-and-lab//



In [ ]:
#define online filepath (aka url):
url='https://geodacenter.github.io/data-and-lab//data/election.zip'

#define local filepath:
local = '/elections.zip'

#download elections data:
urllib.request.urlretrieve(url, local)

#unzip file
shutil.unpack_archive(local, '../../../')

votes = gpd.read_file('/election/election.shp')

In [ ]:
#Let's view the shapefile to get a general idea of the geometry we're looking at:
%matplotlib inline
votes.plot()

In [ ]:
#View the first few line]s of the dataset
votes.head()

The goal of our model is to predict a change in county-level voting trends between the 2012 and 2016 US presidential elections. First, we created a new column indicating the percent change in democratic vote. Then, we plotted the percent democratic vote in 2012, the percent democratic vote in 2016, and the percent change in democratic vote as maps and using global histograms.

In [ ]:
# Add new column pct_dem_change to votes:
votes['pct_dem_change'] = votes['pct_dem_12'] - votes['pct_dem_16']

f,ax = plt.subplots(3,2, figsize=(1.6*6 + 1,4.2*3), gridspec_kw=dict(width_ratios=(6,1)))
for i,col in enumerate(['pct_dem_12','pct_dem_16', 'pct_dem_change']):
    votes.plot(col, linewidth=.05, cmap='RdBu', ax=ax[i,0])
    ax[i,0].set_title(['2012','2016', '2012-2016 change'][i] + "% democratic vote")
    ax[i,0].set_xticklabels('')
    ax[i,0].set_yticklabels('')
    sns.kdeplot(votes[col].values, ax=ax[i,1], vertical=True, shade=True, color='slategrey')
    ax[i,1].set_xticklabels('')
    ax[i,1].set_ylim(-1,1)
f.tight_layout()
plt.show()

### Spatial Autocorrelation
From https://www.sciencedirect.com/topics/computer-science/spatial-autocorrelation

>"Spatial autocorrelation is the term used to describe the presence of *systematic* spatial variation in a variable and positive spatial autocorrelation, which is most often encountered in practical situations, is the tendency for areas or sites that are close together to have similar values."

##**Task 1: Is there evidence for *spatial autocorrelation* in the three plots above? Explain why or why not.**



## Was the county wide percent change in democratic vote related to per capita income?
The next question is how can we use robust statistics to determine whether per capita income or the county land area were related to a change in 2016 voting preferences. To do this, we're going to conduct a linear regression relating our parameters pct_dem_change to INC910213 and ALAND. Then, we're going to use the confidence interval around beta hat (our slope parameter estimate) to determine whether the relationships between INC910213, ALAND, and pct_dem_change are significantly different than zero.

First we're going to visualize how these variables relate in the global data:

In [ ]:
votes.dropna(subset=['pct_dem_12','pct_dem_16'], inplace=True)

In [ ]:
f,ax = plt.subplots(1,4, figsize=(4*3.1,4))

votes[['pct_dem_12','pct_dem_16']].plot.scatter('pct_dem_12','pct_dem_16', ax=ax[0])
ax[0].set_xlabel('2012 Two Party Vote (% Dem)')
ax[0].set_ylabel('2016 Two Party Vote (% Dem)')
print(np.corrcoef(votes['pct_dem_12'].values, votes['pct_dem_16'].values)[0,1])


votes[['INC910213','pct_dem_change']].plot.scatter('INC910213','pct_dem_change', ax=ax[1])
ax[1].set_xlabel('Per capita income')
ax[1].set_ylabel('% Change in Democratic Vote')
print(np.corrcoef(votes['pct_dem_change'].values, votes['INC910213'].values)[0,1])

votes[['ALAND','pct_dem_change']].plot.scatter('ALAND','pct_dem_change', ax=ax[2])
ax[2].set_xlabel('County land area')
ax[2].set_ylabel('% Change in Democratic Vote')
print(np.corrcoef(votes['pct_dem_change'].values, votes['ALAND'].values)[0,1])

votes[['INC910213','ALAND']].plot.scatter('INC910213','ALAND', ax=ax[3])
ax[3].set_xlabel('Per capita income')
ax[3].set_ylabel('County land area')
print(np.corrcoef(votes['INC910213'].values, votes['ALAND'].values)[0,1])

f.tight_layout()
plt.show()

The first scatterplot compares the 2012 percent democratic vote to the 2016 percent democratic vote. The second two plots compare our predictor (2012-2016 % democractic vote, or pct_dem_change) to our two predictor variables (per capita income and county land area). In the fourth plot, we compare our two predictors (per capita income to county land area). We want to establish a trendline in the fourth plot using linear regression, and determine of the slope in that trendline is statistically significant.

###Theret are some features of this data that make it poorly suited for linear regression:

We are told above that we are going to be regressing the 2012-2016 change in percent democratic vote against per capita income and land area. The conditions that we need to meet for linear regression to be applicable are:
1. Linear relationship: *it appears that the relationship is approximately inverse linear, though a linear fit might not be perfect*
2. Multivariate normality: *this might be an issue...the variance in percent change democratic vote appears to increase as a function of decreasing income. This might mean that the residuals are non-normal*
3. No or little multicollinearity: *The correlation coefficient between our two predictors (per capita income and county land area) is below 0.7. For the purposes of this example, paramters have been selected that do not exhibit collinearity. Note: this does not mean the parameters are *independent*, just that the are not linearly related.*
4. No auto-correlation: *We will evaluate this later*
5. Homoscedasticity: *We will evaluate this later*


## Do we have spatial autocorrelation in our data?
When we're looking at distributions of voting preferences, remember that we're aggregating these numbers over arbitrary (er...political) geographic regions.

Each column in that dataframe represents a data value summarized over a US county, but US counties have widely different land areas and populations:

In [ ]:
f,ax = plt.subplots(1,2, figsize=(2*3*1.6, 2))
sns.kdeplot(votes['ALAND'].values, shade=True, color='slategrey', ax=ax[0])
ax[0].set_xlabel('County Land Area')

sns.kdeplot(votes['PST045214'].values, shade=True, color='slategrey', ax=ax[1])
ax[1].set_xlabel('2014 County Population')

Our *spatial sampling rate* (by county) may not accurately represent the spatial frequency of variability in voting preferences and/or income.

First, let's focus on the spatial componnet: the fact that these counties are different sizes.

If we want to identify spatial autocorrelation in our data, we need to first understand how this spatial autocorrelation decays as a function of distance.

To do this, we calculate the Moran's I statistic, which you can think of as the "slope" that we'd get when we regress data values for all geographic entities with data values that neighbor within a given distance. Lets look at our data in lat/lon space again:

In [ ]:
votes.crs

### The horizontal unit (unit of distance) in our votes data is meters.

One issue is that we're sampling spatially at a distinct, and heterogenous, granulatiry. The smallest unit of measurement available in our dataset is the county level. Counties are different sizes. How can we evaluate whether this spatial sampling granularity is of sufficient resolution to capture the scale of variability in our dataset?

* If we are sampling at too course of a spatial scale, we run the risk of missing key patterns of variability in our data (**UNDERSAMPLING**)

* If we are sampling at too fine of a sptial scale, we run the risk of violating assumptions of independence between our individual observations (**OVERSAMPLING**)

The county spatial unit represents a method for discretizing population-level opinions. Some counties may contain communities with very different voting patterns, which we won't be able to resolve with the data (undersampling). Especailly in rural areas, some counties may represent very few people, and could therefore represent an incidence of oversampling.

## Calculating a weights matrix:
The first thing we want to tackle is a quantification of any spatial autocorrelation in our dataset. Spatial autocorrelation inflates our theoretical number of samples (N), artificially increasing the power in our test statistics. In other words, when we're calculating test statistics, spatial autocorrelation in our data can make it seem like parameters that are unimportant are actually significant.

Since we're dealing with a heterogeneous sampling grid in our data, the first thing we want to do is calculate a weights matrix.

We're going to use the Queen function in pysal to do this. Full documentation here: https://pysal.org/libpysal/generated/libpysal.weights.Queen.html

Or just use the built in help with the function below:

In [ ]:
#Calculate weights object
weights = Queen.from_dataframe(votes)

#Use built in plot function to visualize how the weights matrix works
plot_spatial_weights(weights, votes)
plt.show()

The verticies in this plot represent two things:
* First, they link "neigbors" based on the model parameters we set for defining neighborhood (here we use the defaul settings and consider any contiguous polygons).
* The length of the verticies indicates the distance between the centers of neighborhing cells.

"Neighbors" that are father matter less than "neighbors" that are closer in identifying the strength of spatial autocorrelation.

## Calculate Moran's I:
Moran's I statistic quantifies the spatial autocorrelation in your data. From https://en.wikipedia.org/wiki/Moran%27s_I
>"Spatial dependency leads to the spatial autocorrelation problem in statistics since, like temporal autocorrelation, this violates standard statistical techniques that assume independence among observations. For example, regression analyses that do not compensate for spatial dependency can have unstable parameter estimates and yield unreliable significance tests. Spatial regression models (see below) capture these relationships and do not suffer from these weaknesses. It is also appropriate to view spatial dependency as a source of information rather than something to be corrected."

In [ ]:
# calculate Moran and plot
moran_votes = Moran(votes['pct_dem_change'], w=weights)
plot_moran(moran_votes, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
# calculate Moran and plot
moran_income = Moran(votes['INC910213'], w=weights)
plot_moran(moran_income, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
# calculate Moran and plot
moran_land = Moran(votes['ALAND'], w=weights)
plot_moran(moran_land, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
print(moran_votes.p_norm)
print(moran_income.p_norm)
print(moran_land.p_norm)

##Task 2: what do the p_norm values tell us? What hypothesis is being tested, what is the null hypothesis, and do we accept or reject the null hypothesis?


## Task 3: is there autocorrelation in the data? If so, which features exhibit autocorrelation?

## Caluculate a linear regression on the global data:
In this next step, we're going to calculate a linear regression in our data an determine whether that analysis determines a statistically significant relationship between our percent income and percent change in democratic vote.

In [ ]:
#first, forumalate the model.
#isolate and standardize the preditors:
X_raw = np.array(votes[['INC910213', 'ALAND']].values)
X = standardize(X_raw, columns=[0, 1])


#isolate and standardize predictand:
Y = np.array(votes['pct_dem_change'].values)
Y= standardize(Y)

lm = OLS(Y,X)
lm_results = OLS(Y,X).fit().summary()

In [ ]:
print(lm_results)


With p-values of effectively zero, we can reject the null hypothesis that btoh the coefficient on INC910213 (x2) and ALAND (x2) are zero, and conclude that there is a statistically significant relationship. Increasing income and increasing land area are both associated with a statistically significant decrease in percent democratic vote. It is notable that the standard error on the standardized coefficient for both variables are nearly an order of magnitude smaller than the coefficient itself

Now, let's plot our residuals to see if there are any spatial patterns in them.

Remember residuals = predicted - fitted values

In [ ]:
#Add model residuals to our "votes" geopandas dataframe:
votes['lm_resid']=lm.fit().resid

Remember, in OLS regression we depend out our residuals being normally distributed:

In [ ]:
sns.kdeplot(votes['lm_resid'].values, shade=True, color='slategrey')
ax[1].set_xlabel('OLS residuals')

In [ ]:
#Plot them in space:
f, ax1 = plt.subplots(figsize=(10, 6))
votes.plot('lm_resid', ax=ax1, linewidth=.05, cmap='RdBu',legend = True)

These are very not normal residuals. What's going on?

## Task 5: What does a positive residual mean here (the model overpredicted change in democratic vote, the model underpredicted change in democratic vote)?



With spatial data, we also care about spatial autocorrelation of our *residuals*:

In [ ]:
# calculate Moran and plot
moran = Moran(votes['lm_resid'], w=weights)
plot_moran(moran, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
moran.p_norm

## Task 6: Do we have spatially autocorrelated residuals? Provide numbers to back your claim. Why do we care?


In [ ]:
f,ax = plt.subplots(1,3, figsize=(3*3*1.6, 2))

votes[['INC910213','lm_resid']].plot.scatter('INC910213','lm_resid', ax=ax[0])
ax[0].set_xlabel('county income')
ax[0].set_ylabel('linear model residuals')

votes[['ALAND','lm_resid']].plot.scatter('ALAND','lm_resid', ax=ax[1])
ax[1].set_xlabel('county land area')
ax[1].set_ylabel('linear model residuals')

votes[['pct_dem_change','lm_resid']].plot.scatter('pct_dem_change','lm_resid', ax=ax[2])
ax[2].set_xlabel('2012-2016 change in % democratic vote')
ax[2].set_ylabel('linear model residuals')


*The first two plots show heteroskedasticity in our residuals. The second plot shows bias in our model*


## Autocovariate regression: spatial lag model
Let's see if we can get different answers by accounting for our residuals in our model. First, we'll try a spatial lag model. A spatial lag model is a type of autocovariate model that assumes that dependencies exist directly among the levels of the dependent variable, and models them as an "autocovariate". So we create an autocovariate function that describes the degree to which the percent change in democratic vote at one location is affected by the percent change in democratic vote at the nearby locations. The coefficient and p-value for the autocovariate function are interpreted as for the independent variables.

In [ ]:
?spreg.ML_Lag

In [ ]:
Yl = Y.T
Yl.shape = (len(Y),1)

lag=spreg.ML_Lag(Yl, X, weights, name_x=["income", "area"],name_y="vote", name_w="weights")
print(lag.summary)


In [ ]:
#Add model residuals to our "votes" geopandas dataframe:
votes['slm_resid']=lag.u
sns.kdeplot(votes['slm_resid'].values, shade=True, color='slategrey')
ax[1].set_xlabel('SLM residuals')

In [ ]:
# calculate Moran and plot
moran = Moran(votes['slm_resid'], w=weights)
plot_moran(moran, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
#Plot them in space:
f, ax1 = plt.subplots(figsize=(10, 6))
votes.plot('slm_resid', ax=ax1, linewidth=.05, cmap='RdBu',legend = True)

In [ ]:
# calculate Moran and plot
moran = Moran(votes['slm_resid'], w=weights)
plot_moran(moran, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
moran.p_norm

In [ ]:
f,ax = plt.subplots(1,3, figsize=(3*3*1.6, 2))

votes[['INC910213','slm_resid']].plot.scatter('INC910213','slm_resid', ax=ax[0])
ax[0].set_xlabel('Per capita income')
ax[0].set_ylabel('spatial lag model residuals')

votes[['ALAND','slm_resid']].plot.scatter('ALAND','slm_resid', ax=ax[1])
ax[1].set_xlabel('county land area')
ax[1].set_ylabel('spatial lag model residuals')

votes[['pct_dem_change','slm_resid']].plot.scatter('pct_dem_change','slm_resid', ax=ax[2])
ax[2].set_xlabel('2012-2016 change in % democratic vote')
ax[2].set_ylabel('spatial lag model residuals')

## Task 7: When we account for spatial autocorrelation in our data using the spatial lag model, do we still see a significant relationship between our response and predictor variable?





## Task 8: Compare these two models. Do you believe that there is a linear relationship between percent change in democratic vote and income level? What about between change in democratic vote and land area? Explain your interpretation using model parameters and outputs.

